In [ ]:
import pandas as pd
import requests
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Category20, cividis
from bokeh.models import Legend, HoverTool, PrintfTickFormatter, ColumnDataSource, Range1d
from statistics import median
import numpy as np
import datetime
output_notebook()

In [ ]:
# def graph_inflation_of_countries(list_of_countries:list):
#     abbr = [countries[x] for x in list_of_countries] # countries is from imf 
#     inflation_rates = rename_from_abbr_to_full_name(get_all_inflation_rate(abbr))
#     p = figure(title="Inflation Over Time in Non-CFA Franc Zone", 
#            x_axis_label="Year", 
#            y_axis_label="Inflation Rate"
#         )
#     # Define the colors for each country
#     if len(inflation_rates) > 20:
#         colors = cividis(len(inflation_rates))
#     else:
#         colors = Category20[len(inflation_rates)]
#     # Add a line for each country
#     for i, (country, values) in enumerate(inflation_rates.items()):
#         x = list(values.keys())
#         y = list(values.values())
#         p.line(x=x, y=y, color=colors[i], alpha = 0.5)
    
#     # median is the median inflation rate of cfa countries 
#     p.line(x=list(medians.keys()), y = list(medians.values()), color = 'cornflowerblue', line_width = 4,  line_dash = "dashed")
#     hover = HoverTool(tooltips=[("Country", "$name"), ("Year", "@x"), ("Inflation", "@y")], formatters={"@name": "printf", "@x": "numeral", "@y": "numeral"})
#     p.add_tools(hover)
    
#     legend = Legend(items=[(country, [line]) for country, line in zip(inflation_rates.keys(), p.renderers)], location="top_right")
#     p.add_layout(legend)
    
#     # Disable scientific notation for y-axis ticks
#     p.yaxis[0].formatter = PrintfTickFormatter(format="%d")
#     p.legend.title = "Countries"
#     p.legend.background_fill_alpha = 0
#     p.legend.border_line_alpha = 0
#     p.legend.click_policy="hide"
    
#     p.xgrid.grid_line_color = None
#     p.ygrid.grid_line_color = None
#     p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
#     p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
#     p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
#     p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
#     # Show the plot
#     return p

In [ ]:
""" 
TODO:
- fix hover tool
- make lines thicker? 
- turn functions into package
- break out the cfa into the two economic unions?
- check when the metrics were published - found in indicator dict

"""

In [ ]:
"""
NOTES:
- will not remove outliers for cfa franc zone because each period only has 14 data points 

"""

The CFA franc zone consists of 14 countries in sub-Saharan Africa, each affiliated with one of two monetary unions. Benin, Burkina Faso, Côte D’Ivoire, Guinea-Bissau, Mali, Niger, Senegal, and Togo comprise the West African Economic and Monetary Union, or WAEMU, founded in 1994 to build on the foundation of the West African Monetary Union, founded in 1973. The remaining six countries — Cameroon, Central African Republic, Chad, Republic of Congo, Equatorial Guinea, and Gabon — comprise the Central African Economic and Monetary Union, or CAEMC. 

In [ ]:
# all_non_cfa = ['Algeria',
#  'Angola',
#  'Botswana',
#  'Burundi',
#  'Cabo Verde',
#  'Comoros',
#  'Djibouti',
#  'Egypt',
#  'Eritrea',
#  'Eswatini',
#  'Ethiopia',
#  'Gambia, The',
#  'Ghana',
#  'Guinea',
#  'Kenya',
#  'Lesotho',
#  'Liberia',
#  'Libya',
#  'Madagascar',
#  'Malawi',
#  'Mauritania',
#  'Mauritius',
#  'Morocco',
#  'Mozambique',
#  'Namibia',
#  'Nigeria',
#  'Rwanda',
#  'São Tomé and Príncipe',
#  'Seychelles',
#  'Sierra Leone',
#  'Somalia',
#  'South Africa',
#  'South Sudan, Republic of',
#  'Sudan',
#  'Tanzania',
#  'Tunisia',
#  'Uganda',
#  'Zambia',
#  'Zimbabwe',
#  'Congo, Dem. Rep. of the']

In [ ]:
cfa_franc_zone = ['Benin', 'Burkina Faso', 'Cameroon', 'Central African Republic', 'Chad', 'Congo, Republic of ', 'Côte d\'Ivoire', 'Equatorial Guinea', 'Gabon', 'Guinea-Bissau', 'Mali', 'Niger', 'Senegal', 'Togo']

In [ ]:
chunk1_non_cfa = [
    'Algeria',
    'Botswana',
    'Burundi',
    'Cabo Verde',
    'Comoros',
    'Djibouti',
    'Egypt',
    'Eritrea',
    'Eswatini',
    'Ethiopia',
    'Gambia, The',
    'Ghana',
    'Guinea',
    'Kenya',
    'Lesotho',
    'Liberia',
    'Libya',
    'Madagascar',
    'Malawi',
    'Mauritania',
    'Mauritius',
    'Morocco',
    'Mozambique',
    'Namibia',
    'Nigeria',
    'Rwanda'
]

chunk2_non_cfa = [
    'São Tomé and Príncipe',
    'Seychelles',
    'Sierra Leone',
    'Somalia',
    'South Africa',
    'South Sudan, Republic of',
    'Sudan',
    'Tanzania',
    'Tunisia',
    'Uganda',
    'Zambia',
    'Zimbabwe',
    'Angola', 
    'Congo, Dem. Rep. of the'
]

In [ ]:
def rename_from_abbr_to_full_name(inflation_dict):
    abbrv = list(inflation_dict.keys())
    for key in abbrv:
        inflation_dict[all_countries['countries'][key]['label']] = inflation_dict.pop(key)
    return inflation_dict

In [ ]:
def get_data_from_imf(url: str):
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data
        data = response.json()
        # Do something with the data
        return data
    else:
        print("Error: Could not retrieve data from the API endpoint.")

In [ ]:
def get_all_metric_data(country_list, metric_abbr):
    abbr = [countries[x] for x in country_list]
    url = f"https://www.imf.org/external/datamapper/api/v1/{metric_abbr}"
    for country in abbr:
        url += f'/{country}'
    return get_data_from_imf(url)['values'][metric_abbr]

In [ ]:
def find_outliers_IQR(df):
     # Calculate the first quartile (Q1) and third quartile (Q3)
    q1=df.quantile(0.25, interpolation = 'midpoint', numeric_only=True)
    q3=df.quantile(0.75, interpolation = 'midpoint', numeric_only=True)
    
    # Calculate the interquartile range (IQR)
    IQR=q3-q1
    
    # Calculate the lower and upper bounds for outliers
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
    return outliers

In [ ]:
def remove_outliers(original_df, outlier_df):
    """40 countries per year in the noncfa, so i think its fine to remove outliers"""
    original_df[outlier_df.notna()] = np.nan
    return original_df

In [ ]:
def generate_median_df(df):
    median_df = df.median().to_frame(name="median")
    median_df.index.name="year"
    median_df.reset_index(inplace=True)
    median_df.sort_values(by='year', inplace=True)
    median_df['year'] = median_df['year'].astype(int)  # had an issue where some of the data was returning as string 
    median_df['median'] = median_df['median'].astype(float) # had an issue where some of the data was returning as string 
    return median_df

In [ ]:
def remove_future_years(median_df):
    current_year = datetime.datetime.now().year # drop years that dont exist yet
    return median_df[median_df['year'] <= current_year]

In [ ]:
def generate_graph(cfa_df, non_cfa_df, metric_name):
    p = figure(
            title=f"{metric_name} over time", 
            x_axis_label="Year", 
            y_axis_label=metric_name,
            width=1000, 
            height=400,
        )
    p.line(x='year', y = 'median', color = 'rosybrown', line_width = 2, legend_label=f"Non-CFA Median {metric_name}", source = ColumnDataSource(non_cfa_df))
    p.line(x='year', y = 'median', color = 'cornflowerblue', line_width = 2, legend_label=f"CFA Median {metric_name}", source = ColumnDataSource(cfa_df))
    for legend in p.legend:
        p.add_layout(legend, 'right')
    # hover = HoverTool(tooltips=[("Year", "@x"), (metric_name, "@y")], formatters={"@x": "numeral", "@y": "numeral"})
    # p.add_tools(hover)
    # p.yaxis[0].formatter = PrintfTickFormatter(format="%d")
    p.legend.click_policy="hide"
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
    return p

In [ ]:
def generate_metric_graphs(indicators_imf_dict):
    for indicator in indicators_imf_dict['indicators']: 
        try:
            chunk_1_data_non_cfa = rename_from_abbr_to_full_name(get_all_metric_data(chunk1_non_cfa, indicator))
            chunk_2_data_non_cfa = rename_from_abbr_to_full_name(get_all_metric_data(chunk2_non_cfa, indicator))
            chunk_1_data_non_cfa.update(chunk_2_data_non_cfa)
            non_cfa_df = pd.DataFrame.from_dict(chunk_1_data_non_cfa).T
            non_cfa_df = remove_outliers(non_cfa_df, find_outliers_IQR(non_cfa_df))
            cfa_data =  rename_from_abbr_to_full_name(get_all_metric_data(cfa_franc_zone, indicator))
            cfa_df = pd.DataFrame.from_dict(cfa_data).T
            median_cfa_df = remove_future_years(generate_median_df(cfa_df))
            median_non_cfa_df = remove_future_years(generate_median_df(non_cfa_df))
            p = generate_graph(median_cfa_df, median_non_cfa_df, indicators['indicators'][indicator]['label'])
            show(p)
        except:
            print(f"issue with indicator {indicator}")

In [ ]:
all_countries = get_data_from_imf("https://www.imf.org/external/datamapper/api/v1/countries")
countries = {v['label']: k for k, v in all_countries['countries'].items()}

In [ ]:
indicators = get_data_from_imf("https://www.imf.org/external/datamapper/api/v1/indicators")

In [ ]:
generate_metric_graphs(indicators)